![LogoUC3M](https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png)

Aprendizaje Automático · Grado en Ingeniería Informática · Curso 2022/23
---
**Cuaderno 2** - Práctica 1: Predicción de la producción de energía solar
<br><br>
***Grupo Reducido 82 - Grupo de laboratorio 13***


## **Entrenamiento modelo final**
Una vez seleccionado y evaluado el método, **entrenamos** el método y lo guardamos en un fichero *modelo_final.pkl*.

In [ ]:
# Datos disponibles.
disp_df = pd.read_csv("disp_st13ns1.txt.bz2",
                      compression="bz2",
                      index_col=0)

# Datos competición.
comp_df = pd.read_csv("comp_st13ns1.txt.bz2",
                      compression="bz2",
                      index_col=0)

In [ ]:
# Creación del modelo final.
final_clf = tree.DecisionTreeClassifier()
np.random.seed(13)

# Entrenamos el modelo final.
final_clf.fit(X, y)

In [ ]:
from joblib import dump, load

# Guardamos el modelo final en un archivo.
dump(final_clf, 'modelo_final.pkl')

# Cargamos el modelo desde el archivo.
final_clf_reloaded = load('modelo_final.pkl')

## **Predicciones**
A continuación, utilizaremos el modelo final para obtener **predicciones** para el conjunto de datos de la competición. Además, guardaremos estas predicciones en un fichero llamado *predicciones.csv*.